In [1]:
import random
from datetime import datetime
from tqdm import tqdm
import numpy as np
from numpy.linalg import norm, det, eigvals
from scipy.linalg import expm

random.seed(42)

### Экспонента симметрической матрицы 3 x 3. См. [статью](https://www.jstage.jst.go.jp/article/ipsjjip/23/2/23_171/_pdf).

Общий вид симметрической матрицы:

$$
\begin{pmatrix}
a & b & c\\
b & d & e\\
c & e & f
\end{pmatrix}
$$


In [10]:
def custom_expm_3d(mat):
    l1, l2, l3 = compute_eigenvalues(mat)
    x, y, z = compute_coefs(l1, l2, l3)
    mat_exp = x * mat @ mat + y * mat + z * np.eye(3)
    return mat_exp

def compute_coefs(l1, l2, l3):
    if (np.abs(l1 - l2) < 1e-6) and (np.abs(l1 - l3) < 1e-6) and (np.abs(l2 - l3) < 1e-6):
        x, y, z = 0, 0, np.exp(l1)
    elif (np.abs(l1 - l2) < 1e-6) or (np.abs(l2 - l3) < 1e-6) or (np.abs(l1 - l3) < 1e-6):
        if np.abs(l1 - l2) < 1e-6:
            l_eq, l_not_eq = l1, l3
        elif np.abs(l2 - l3) < 1e-6:
            l_eq, l_not_eq = l2, l1
        else:
            l_eq, l_not_eq = l1, l2
        s = np.exp(l_eq) / (l_eq - l_not_eq)
        t = np.exp(l_not_eq) / (l_eq - l_not_eq)
        x, y, z = 0, s - t, t * l_eq - s * l_not_eq
    else:
        s = np.exp(l1) / ((l1 - l2) * (l1 - l3))
        t = np.exp(l2) / ((l2 - l3) * (l1 - l2))
        u = np.exp(l3) / ((l2 - l3) * (l3 - l1))
        x = s + t + u
        y = -s * (l2 + l3) + t * (l3 + l1) + u * (l1 + l2)
        z = s * l2 * l3 - t * l3 * l1 - u * l1 * l2
    return x, y, z

def compute_eigenvalues(mat):
    mat_det = det(mat) 
    fro_norm = norm(mat, 'fro')
    mat_trace = np.trace(mat)
    p = (mat_trace ** 2 - 3 * fro_norm ** 2) / 6
    q = (5 / 54 * mat_trace ** 3) - (1 / 6 * mat_trace * fro_norm ** 2) - mat_det
    r = np.sqrt(-12 * p)
    k = -108 * q / r ** 3

    l1 = (r * np.cos(np.arccos(k) / 3) + mat_trace) / 3
    l2 = (r * np.cos((np.arccos(k) + 2 * np.pi) / 3) + mat_trace) / 3
    l3 = (r * np.cos((np.arccos(k) + 4 * np.pi) / 3) + mat_trace) / 3

    return l1, l2, l3

Рассмотрим для примера симметричную матрицу

$$
\begin{pmatrix}
1 & 2 & 3\\
2 & 4 & 5\\
3 & 5 & 6
\end{pmatrix}
$$

In [11]:
mat = np.array([[1, 2, 3], [2, 4, 5], [3, 5, 6]])

expm(mat)

array([[ 9093.56594553, 16384.33614782, 20431.36731548],
       [16384.33614782, 29524.933261  , 36815.7034633 ],
       [20431.36731548, 36815.7034633 , 45909.26940883]])

In [12]:
custom_expm_3d(mat)

array([[ 9095.84277295, 16388.40191108, 20436.40886191],
       [16388.40191108, 29532.25163486, 36824.81077298],
       [20436.40886191, 36824.81077298, 45920.65354593]])

Будем сравнивать скорость работы `scipy` реализации и нашей на 1_000_000 сгенерированных матриц.

In [5]:
synthetic_data_3d = []

for _ in range(1_000_000):
    coefs = []
    for _ in range(6):
        coefs.append(random.randint(1, 100))
    a, b, c, d, e, f = coefs
    symm_mat = np.array([[a, b, c], [b, d, e], [c, e, f]])
    synthetic_data_3d.append(symm_mat)

Для начала сравним `numpy` реализацию поиска собственных векторов и нашу.

In [6]:
scipy_mat_eigvals_3d_list = []

start_time = datetime.now()
for sym_mat in tqdm(synthetic_data_3d):
    scipy_mat_eigvals_3d_list.append(np.sort(eigvals(mat)))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для Scipy реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:08<00:00, 123711.02it/s]

Время для Scipy реализации: 8.1 секунд. 


In [7]:
custom_mat_eigvals_3d_list = []

start_time = datetime.now()
for sym_mat in tqdm(synthetic_data_3d):
    custom_mat_eigvals_3d_list.append(np.sort(compute_eigenvalues(mat)))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для кастомной реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:10<00:00, 94475.63it/s]

Время для кастомной реализации: 10.59 секунд. 


По времени кастомной реализация получается даже чуть хуже

In [8]:
eigvals_diff = []

for scipy_eigvals, custom_mat_exp in zip(scipy_mat_eigvals_3d_list, custom_mat_eigvals_3d_list):
    eigvals_diff.append(norm(scipy_eigvals - custom_mat_exp))

print(f'L2 норма разности собтственных векторов, полученных Scipy и кастомной реализациями {round(np.mean(eigvals_diff), 5)}')

L2 норма разности собтственных векторов, полученных Scipy и кастомной реализациями 0.0


Приближаем `numpy` реализацию идеально

Теперь сравним сами функции вычисления матричной экспоненты.

In [9]:
scipy_mat_exp_3d_list = []

start_time = datetime.now()
for sym_mat in tqdm(synthetic_data_3d):
    scipy_mat_exp_3d_list.append(expm(mat))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для Scipy реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:48<00:00, 20451.94it/s]

Время для Scipy реализации: 48.9 секунд. 


In [10]:
custom_mat_exp_3d_list = []

start_time = datetime.now()
for sym_mat in tqdm(synthetic_data_3d):
    custom_mat_exp_3d_list.append(custom_expm_3d(mat))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для кастомной реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:17<00:00, 56880.74it/s]

Время для кастомной реализации: 17.58 секунд. 


In [11]:
max_exp_diff_l2 = []
max_exp_diff_fro = []

for scipy_mat_exp, custom_mat_exp in zip(scipy_mat_exp_3d_list, custom_mat_exp_3d_list):
    max_exp_diff_l2.append(norm(scipy_mat_exp - custom_mat_exp))
    max_exp_diff_fro.append(norm(scipy_mat_exp - custom_mat_exp, 'fro'))

print(f'L2 норма разности матричных экспонент, полученных Scipy и кастомной реализациями {round(np.mean(max_exp_diff_l2), 5)}')
print(f'Норма Фробениуса разности матричных экспонент, полученных Scipy и кастомной реализациями {round(np.mean(max_exp_diff_fro), 5)}')

L2 норма разности матричных экспонент, полученных Scipy и кастомной реализациями 20.93138
Норма Фробениуса разности матричных экспонент, полученных Scipy и кастомной реализациями 20.93138
